#  Неробастный и робастный t-тест: общий случай

In [25]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
from scipy.stats import t # t-распределение и критические значения

# Не показывать FutureWarnings
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [26]:
# импорт данных
df = pd.read_csv('Labour.csv')
df.head()

,capital,labour,output,wage
0,2.606563,184,9.250759,43.080307
1,1.323237,91,3.664310,27.780016
2,22.093692,426,28.781516,44.467748
3,10.737851,72,4.124642,39.734710
4,1.161365,46,2.890150,34.650709


Рассмотрим регрессию

$$
	\log(output)=\beta_0+\beta_1\log(capital)+\beta_2\log(labour)+\beta_3\log^2(capital)+\beta_4\log^2(labour)+error
$$

Подгоним модель методом OLS с двумя опциями
* с оценкой ковариационной матрицы коэффициентов по умолчанию (неробастной) $Var_{OLS}(\hat{\beta})=s^2(X^\top X)^{-1}$, где $s^2=\sum e^2_i/(n-k-1)$
* с оценкой ковариационной матрицы коэффициентов, робастной к гетероскедастичности (HC3-оценкой)

Далее CME = Covariance Matrix Estimator

Стандартные ошибки коэффициентов $s.e.(\beta)$ - корни из диагональных элементов матрицы CME

In [27]:
# спецификация модели
mod = smf.ols(formula='np.log(output)~np.log(capital)+np.log(labour)+I(np.log(capital)**2)+I(np.log(labour)**2)', data=df)
# подгонка модели с ковариационной матрицей по умолчанию (неробастной)
res_ols = mod.fit()
# подгонка модели с робастной HC3 ковариационной матрицей
res_hc = mod.fit(cov_type='HC3')

## Гипотеза
Тестируется гипотеза $H_0:\beta_{4}=0.04$

Пусть уровень значимости $\alpha=5\%=0.05$

Критическое значение t-распределения

In [28]:
# уровнь значимости
sign_level = 0.05
# критическое значение t-распределения
t.ppf(q=1-sign_level/2, df=mod.df_resid)

1.9641790265687167

## Результаты неробастного t-теста

In [29]:
res_ols.t_test('I(np.log(labour) ** 2)=0.04')

<class 'statsmodels.stats.contrast.ContrastResults'>
                             Test for Constraints                             
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
c0             0.0203      0.010     -2.057      0.040       0.001       0.039

**ВЫВОД**: На уровне значимости 5% коэффициент при регрессоре $log^2(labour)$ значимо отличается от числа 0.04

## Результаты робастного t-теста (HC3-оценка ковариационной матрицы)

In [30]:
res_hc.t_test('I(np.log(labour) ** 2)=0.04')

<class 'statsmodels.stats.contrast.ContrastResults'>
                             Test for Constraints                             
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
c0             0.0203      0.021     -0.940      0.347      -0.021       0.061

**ВЫВОД**: На уровне значимости 5% коэффициент при регрессоре $log^2(labour)$ незначимо отличается от числа 0.04